In [1]:
# include should only be used once, serves as copy paste code purpose
# using should be used whenever contents in this module is called
# in individual modules, using other modules should consider hierarcy, 
# which is also related to how files are included in MAIN.

include("../src/Config_files/ConfigDataType.jl")
include("../src/SpatialAlgebra.jl")
include("../src/ConstructSystem.jl")
include("../src/UpdateSystem.jl")
include("../src/TimeMarching.jl")

using ConfigDataType
using SpatialAlgebra
using ConstructSystem
using UpdateSystem
using TimeMarching

In [2]:
# body = config_body(4,4,[1. 2. 3.;4. 5. 6.],1.)
nbody = 4
ndim = 2
config_body = ConfigBody(nbody)

 nbody=4
 nverts=4
 verts=[0.0 0.0; 1.0 0.0; 1.0 0.25; 0.0 0.25]
 ρ=0.01


In [3]:
njoint = nbody
config_joints = Vector{ConfigJoint}(njoint)

# set the first active joint
qJ_init = 
active_motion = Motions("oscillatory", [π/4, 1., 0.])
active_dof = Dof(3, "active", 0., 0., active_motion)
config_joints[1] = ConfigJoint(njoint, 1, "revolute",
                               zeros(Float64,6), zeros(Float64,6),
                               0, [active_dof], zeros(Float64,6))

# set the rest passive joint
for i = 2:njoint
#     config_joints[i] = deepcopy(config_joints[1])
    config_joints[i] = ConfigJoint(njoint, "revolute")
    config_joints[i].body1 = i-1
end

In [4]:
bodys = Vector{SingleBody}(nbody) # body system
for i = 1:nbody
    bodys[i] = AddBody(i, config_body) # add body
end

In [5]:
joints = Vector{SingleJoint}(njoint) # joint system
for i = 1:njoint
    joints[i] = AddJoint(i, config_joints[i]) # add joint
end

In [6]:
# assemble system to a chain
gravity = [0., 0., 0., ] # [0., -9.8, 0., ]
system = System(ndim, nbody, njoint, gravity)
bodys, joints, system = AssembleSystem!(bodys, joints, system)
system

ndim = 2, njoint = 4, nbody = 4
ndof = 24, nudof = 4, ncdof = 20, np = 3, na = 1
udof = [3, 9, 15, 21]
udof_p = [9, 15, 21]
udof_a = [3]
nudof_HERK = 3, ncdof_HERK = 21
udof_HERK = [9, 15, 21]
gravity = [0.0, 0.0, 0.0]
kinmap = [1 1]


In [7]:
# test function UpdatePosition!
bodys, joints, system = UpdatePosition!(bodys, joints, system)

# test function UpdateVelocity!
for i = 1:nbody
    @assert length(bodys[i].v) == 6
    @assert length(bodys[i].Xp_to_b) == 36
    @assert length(joints[i].vJ) == 6
end
v = ones(Float64,24)
bodys, joints, system, vJ = UpdateVelocity!(bodys, joints, system, v)

# do some output
bodys[1]
bodys[2]
joints[1]
system

ndim = 2, njoint = 4, nbody = 4
ndof = 24, nudof = 4, ncdof = 20, np = 3, na = 1
udof = [3, 9, 15, 21]
udof_p = [9, 15, 21]
udof_a = [3]
nudof_HERK = 3, ncdof_HERK = 21
udof_HERK = [9, 15, 21]
gravity = [0.0, 0.0, 0.0]
kinmap = [1 1]


In [8]:
t = 0.0
dt = 1e-3
qJ_total = zeros(Float64,6*system.nbody)
v_total = zeros(Float64,6*system.nbody)
for i = 1:system.nbody
    qJ_total[6i-5:6i] = joints[i].qJ
    v_total[6i-5:6i] = bodys[i].v
end
soln = Soln(t, dt, qJ_total, v_total)


TimeMarching.Soln{Float64}(0.0, 0.001, [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], Float64[], Float64[])

In [9]:
soln, bodys, joints, system = HERK!(soln, bodys, joints, system)

(TimeMarching.Soln{Float64}(0.001, 3.1365169163476076e-5, [0.000788675, 0.000788675, 0.000773172, 0.000788675, 0.000788675, 0.000788675, -1.25968e-20, 6.41111e-20, 0.000177337, -3.23251e-20  …  0.000613752, 4.92484e-20, -0.000197169, 0.000197169, -9.08396e-20, 7.86948e-20, -0.00142401, -1.05861e-19, -0.000197169, 0.000197169], [0.0, 0.0, -0.0310061, 0.0, 0.0, 0.0, 2.44348e-5, -2.44726e-5, 3.59662, 1.84367e-5  …  -1.91275, -0.00212622, 0.894217, -0.000695159, -0.0013004, 0.00130673, 1.77913, -0.000293596, 0.414522, -0.00173057], [1000.0, 1000.0, 968.994, 1000.0, 1000.0, 1000.0, 1000.03, 999.969, -2842.32, 1000.02  …  2932.0, 1000.19, 31.3756, 1000.09, 999.993, 1000.01, 28.9659, 999.876, 514.678, 1000.15], [0.00660834, -4.02037, -0.212565, -15.0055, -1.6912, -8.73772, -0.559938, -3.83199, -11.2518, -0.142229  …  -0.52188, -3.10022, -7.50115, 0.207519, -4.37355, -0.185454, -1.823, -3.75026, -0.0457556, -2.18788]), ConstructSystem.SingleBody[ConstructSystem.SingleBody(1, 0, [2], 1, 4, [0.0

In [13]:
soln.v

24-element Array{Float64,1}:
  0.0        
  0.0        
 -0.0310061  
  0.0        
  0.0        
  0.0        
  2.44348e-5 
 -2.44726e-5 
  3.59662    
  1.84367e-5 
 -0.00777599 
  6.152e-6   
 -0.00280995 
  0.00281426 
 -1.91275    
 -0.00212622 
  0.894217   
 -0.000695159
 -0.0013004  
  0.00130673 
  1.77913    
 -0.000293596
  0.414522   
 -0.00173057 